In [3]:
#Расчёт пластинчатого ТО с прямоугольным  сечением
import matplotlib.pyplot as pt
import numpy as n
from CoolProp.CoolProp import PropsSI as p
from scipy.optimize import root
import math as m

#Исходные данные:
G1 = 298
T11 = 50
T12 = 37
G2 = 298
T21 = 32
T22 = 41
P11 = 243000
P21 = 3300000

R236ea = 'REFPROP::R236ea'







P12 = P11
P22 = P21

##############################################

H11  = p('H','T', T11+273.15, 'P', P11, R236ea)
H12  = p('H','T', T12+273.15, 'P', P11, R236ea)
H21  = p('H','T', T21+273.15, 'P', P21, R236ea)

#Разбиение по энтальпии:
s    = 20 #Кол-во шагов
step = (H11-H12)/(s) 

#Нулевые массивы:
t1   = n.zeros(s+1)
t2   = n.zeros(s+1)
Q    = n.zeros(s+1)

#Противоток:
h11 = H11
i=0
for i in range(s+1):
    t1[i] = p('T','H', h11, 'P', P11, R236ea)-273.15 
    h12 = h11 - step
    dQ = G1 * (h11-h12)
    if i < s:
        Q[i+1]=Q[i]+dQ
    h11 = h12
h21 = H21
i=0
for i in range(s+1):
    t2[s-i] = p('T','H', h21, 'P', P21, R236ea)-273.15  
    h22 = h21+((Q[s-i]-Q[s-i-1])/G2)
    h21 = h22
DT=t1-t2
##############################################

t1av = n.zeros(s)
t2av = n.zeros(s)
ro1 = n.zeros(s)
ro2 = n.zeros(s)
nu1 = n.zeros(s)
nu2 = n.zeros(s)
lambda1 = n.zeros(s)
lambda2 = n.zeros(s)
Pr1 = n.zeros(s)
Pr2 = n.zeros(s)
DTav = n.zeros(s)
w1 = n.zeros(s)
w2= n.zeros(s)
Re1= n.zeros(s)
Re2= n.zeros(s)
Nu1= n.zeros(s)
Nu2= n.zeros(s)
f1= n.zeros(s)
f2= n.zeros(s)
fNu1= n.zeros(s)
fNu2= n.zeros(s)
alpha1= n.zeros(s)
alpha2= n.zeros(s)
K= n.zeros(s)
F= n.zeros(s+1)
L= n.zeros(s+1)
P1= n.zeros(s+1)
P2= n.zeros(s+1)
t_st = n.zeros(s)
Pr_st1 = n.zeros(s)
Pr_st2 = n.zeros(s)


#Для каждого участка:
for i in range(s):
    #Свойства
    t1av[i]   =(t1[i]+t1[i+1])/2
    t2av[i]   =(t2[i]+t2[i+1])/2
    ro1[i]    =p('D','T', t1av[i]+273.15, 'P', P11, R236ea)
    ro2[i]    =p('D','T', t2av[i]+273.15, 'P', P21, R236ea)
    nu1[i]    =p('V','T', t1av[i]+273.15, 'P', P11, R236ea)/ro1[i]
    nu2[i]    =p('V','T', t2av[i]+273.15, 'P', P21, R236ea)/ro2[i]
    lambda1[i]=p('L','T', t1av[i]+273.15, 'P', P11, R236ea)
    lambda2[i]=p('L','T', t2av[i]+273.15, 'P', P21, R236ea)
    Pr1[i]    =p('PRANDTL','T', t1av[i]+273.15, 'P', P11, R236ea)
    Pr2[i]    =p('PRANDTL','T', t2av[i]+273.15, 'P', P21, R236ea)
    
    
    #Стенка
    t_st[i] = (t1av[i]+t2av[i])/2
    Pr_st1[i] = p('PRANDTL','P',P11,'T',t_st[i]+273.15,R236ea)
    Pr_st2[i] = p('PRANDTL','P',P21,'T',t_st[i]+273.15,R236ea)
    
    
    DTav[i]   =t1av[i]-t2av[i]
#От выпадающих Pr:
for i in range(1,s):
    if abs(Pr1[i]-Pr1[i-1])>Pr1[i-1]:
        Pr1[i]=p('PRANDTL','T', t1[i]+273.15, 'P', P11, R236ea)
    if abs(Pr2[i]-Pr2[i-1])>Pr2[i-1]:
        Pr2[i]=p('PRANDTL','T', t2[i]+273.15, 'P', P21, R236ea)

#Геометрия
lambdaw = 65   #теплопроводность металла, коррозионно-стойкая сталь 12Х18Н10Т
t = 0.002     #толщина пластины
wpl = 0.64        #ширина пластины
delta = 10**-5 #Шероховатость
fproh = 0.00368
Per = 1.28
dh = 4*fproh/Per
print('dh', dh)
mas = 19.4
l = 1.932
ml = mas/l #масса единицы длины 

#Геометрия "выхода":
roout1 = p('D','T', t1[s]+273.15, 'P', P11, R236ea)
roout2 = p('D','T', t2[s]+273.15, 'P', P21, R236ea)


wout1 = 28
Fproh1 = G1/(roout1*wout1)
Npl1 = m.ceil(Fproh1/fproh)
Npl2 = m.ceil(Npl1)+1
Nsumm = Npl1+Npl2
Fproh2 = Npl2*fproh
Fl = ((Npl1+Npl2)/2)*Per


P1[0]=P11
P2[0]=P21
#Для каждого участка:
for i in range(s):
    w1[i] = G1/(ro1[i]*Fproh1)
    w2[i] = G2/(ro2[i]*Fproh2)
    Re1[i] = w1[i]*dh/nu1[i]
    Re2[i] = w2[i]*dh/nu2[i]
    Nu1[i] =0.1*Re1[i]**0.73*Pr1[i]**0.43*(Pr1[i]/Pr_st1[i])**0.25
    Nu2[i] = 0.1*Re2[i]**0.73*Pr2[i]**0.43*(Pr2[i]/Pr_st2[i])**0.25
    f1[i] = (6.0/Re1[i])**0.25
    f2[i] = (6.0/Re2[i])**0.25

    alpha1[i] = Nu1[i]*lambda1[i]/dh
    alpha2[i] = Nu2[i]*lambda2[i]/dh
    K[i] = 1/((1/alpha1[i])+(t/lambdaw)+(1/alpha2[i]))
    dF = (Q[i+1]-Q[i])/(K[i]*DTav[i])
    dL = dF/Fl
    if i < s:
        F[i+1]=F[i]+dF
    if i < s:
        L[i+1]=L[i]+dL
    dP1 = f1[i]*dL/dh*ro1[i]*w1[i]**2/2
    dP2 = f2[i]*dL/dh*ro2[i]*w2[i]**2/2
    if i < s:
        P1[i+1]=P1[i]-dP1
        P2[i+1]=P2[i]-dP2
DP1 = P1[0]-P1[s]
DP2 = P2[0]-P2[s]
print(F[-1],Nsumm, L[-1])
print(L[-1], Nsumm*(t+0.006), wpl)
print (alpha1[-1], alpha2[-1])
print ('w1', w1[-1], 'w2', w2[-1])
print ('DP1', DP1/1000, 'DP2', DP2/1000)


#Масса ТО

Massa =  ml * L[-1] * Nsumm 
print ('Масса', Massa, 'кг')

dh 0.0115
727.3079827582046 375 3.030449928159186
3.030449928159186 3.0 0.64
1472.2500544973145 1522.0502849701138
w1 28.03893866553004 w2 0.3041680974839624
DP1 99.95042222938454 DP2 2.4975048097483814
Масса 11411.24390650004 кг
